# Stack Overflow 2025 Developer Survey

La última pregunta del segundo parcial hacía referencia a la encuesta anual (del 2025) que Stack Overflow hace entre desarrolladores. Esta encuesta realmente existe.

Lo primero que vamos a hacer es descargar los datos, disponibles en un archivo comprimido, y descomprimirlos.

In [ ]:
!wget https://survey.stackoverflow.co/datasets/stack-overflow-developer-survey-2025.zip -P data
!unzip data/stack-overflow-developer-survey-2025.zip -d data

El archivo con los resultados es un archivo de texto en formato CSV (*comma-separated values*). Hay varias formas de leerlo en Python, en esta ocasión, vamos a utilizar la biblioteca `pandas`, que nos permite leerlo con una sola instrucción.

In [ ]:
import pandas as pd

data_full = pd.read_csv("data/survey_results_public.csv")

Podemos observar que, efectivamente, contiene más de 49 mil respuestas:

In [ ]:
data_full.info()

Y echarle un ojo al contenido:

In [ ]:
data_full.head()

Son 170 columnas de datos (como que los desarrolladores *de veras* tenían ganas de contestar la encuesta, ¿no?), y no las alcanzamos a ver. 

Tampoco las vamos a poder listar con la propiedad `columns` del `DataFrame` (¡ah! la estructura que nos regresó `pandas` es un `DataFrame`):

In [ ]:
data_full.columns

Pero siempre las podemos imprimir una por una, con nuestro viejo amigo `for`:

In [ ]:
for column in data_full.columns:
    print(column)

Revisándolas, identificamos algunas columnas prometedoras. Veamos.

In [ ]:
data_full[["Country", "Currency", "CompTotal", "LanguageHaveWorkedWith", "ConvertedCompYearly"]].head()

Vamos a quedarnos con las columnas `LanguageHaveWorkedWith` y `ConvertedCompYearly` que parece ser el sueldo anual expresado en dólares (checa la última fila), y los vamos a renombrar para manejarlas con mayor facilidad.

> ***Nota***: Dentro de los datos que descargamos hay un archivo `survey_results_schema.csv`, que, por su nombre, debe contener una descripción de cada una de las columnas.

In [ ]:
mapper = {"LanguageHaveWorkedWith": "languages", "ConvertedCompYearly": "salary"}
data = data_full[["LanguageHaveWorkedWith", "ConvertedCompYearly"]].rename(columns=mapper)
data.sample(10)

Vemos que:

- La columna `languages` tiene una "lista" (que no es una lista de Python) con los lenguajes separados por punto y coma (`;`), y
- Hay muchos valores `NaN`. abreviatura de *not a number*, y que, esencialmente son valores faltantes (siempre no estaban tan entusiasmados por contestar la encuesta).

¿Cuántos faltaran?

In [ ]:
data.info()

De las 49 mil respuestas, solo 31 mil indicaron los lenguajes con los que han trabajado, y 24 mil, su salario.

Vamos a eliminar los registros que tengan datos faltantes.

In [ ]:
data = data.dropna()

Y a ver con qué nos quedamos.

In [ ]:
data.info()

22 mil registros. No está tan mal.

Revisemos la columna `languages`, ¿cuántos valores diferentes tendrá?

In [ ]:
data.languages.unique()

¡11,000 valores diferentes! No es que haya tantos lenguajes (¿o sí), sino las diferentes combinaciones. No nos vamos a detener a analizarlas.

Vamos a convertir la columna `languages` en una verdadera lista de Python.

In [ ]:
data["languages"] = data["languages"].apply(lambda x: x.split(";"))
data.sample(10)

Se ve bien. Veamos a detalle la primera fila:

In [ ]:
print(data["languages"][0])
print(type(data["languages"][0]))

Efectivamente, la columna ya es una lista.

Había pensado escribir el `DataFrame` en un archivo JSON y leerlo para convertirlo en la lista de listas que necesitamos:

In [ ]:
data.to_json("data/salaries.json", orient="values")

Pero `pandas` ya tiene una función que lo hace todo en un paso, usémosla.

In [ ]:
data_list = data.values.tolist()
data_list

¡Listo! Ya tenemos la lista maestra que les describí en el parcial. Hora de analizarla con la función que escribimos.

Voy a copiar mi versión de la solución en la siguiente celda. Ustedes pueden copiar la suya para probarla.

In [ ]:
# Solución a la pregunta 8 del segundo parcial
def analizar(encuesta):
    salarios = {
        "solo python": [],
        "python y otros": [],
        "no python": [],
    }
    for renglon in encuesta:
        lenguajes, salario = renglon
        python = 0
        otros = 0
        for lenguaje in lenguajes:
            if "python" in lenguaje.lower():
                python += 1
            else:
                otros += 1
        if python == 0:
            salarios["no python"].append(salario)
        elif otros == 0:
            salarios["solo python"].append(salario)
        else:
            salarios["python y otros"].append(salario)
    
    promedios = {}
    for categoria in salarios:
        promedios[categoria] = sum(salarios[categoria]) / len(salarios[categoria])

    return promedios

Y vamos a llamar la función con los datos que tenemos. A ver.

In [ ]:
summary = analizar(data_list)
summary

No me gustó el formato. Lo arreglamos:

In [ ]:
for language, salary in summary.items():
    print(f"{language:15} {salary:10,.2f}")

¡Ups! Los que saben solo Python son los que menos ganan. Ya qué.

Ya que andamos en esto, vamos a ver de cuántas formas diferentes escribieron "Python".

In [ ]:
python = set()
for languages, salary in data_list:
    for language in languages:
        if "python" in language.lower():
            python.add(language)
python

Mm. A los que filtraron por que iniciara con "python" se les escapó ese MicroPython. (Les dejo de ejercicio comprobar si los que programan con MicroPython también programan con Python.)

Vamos a hacer más general el filtrado, vamos a filtrar por "py".

In [ ]:
python = set()
for languages, salary in data_list:
    for language in languages:
        if "py" in language.lower():
            python.add(language)
python

OK, nos sale lo mismo.

Oigan, y recordando eso de las once mil combinaciones de lenguajes, ¿de cuántos lenguajes diferentes estamos hablando?

In [ ]:
lenguajes = set()
for languages, salary in data_list:
    for language in languages:
        lenguajes.add(language)
len(lenguajes)

Eso ya está más razonable. ¿Cuáles son?

> ***Reflexión filosófica***: Por cierto, ¿no era 42 la respuesta al sentido de la vida, el universo y todo lo demás?

In [ ]:
lenguajes

***Nota final***: Esto que hicimos de convertir un `DataFrame` de `pandas` en lista para luego analizarlo, no es lo que se hace usualmente. Si lo tenemos en `pandas`, así lo dejamos y analizamos el `DataFrame`, pero ni modo que los pusiera a trabajar con `pandas` en el examen, ¿no?